# 🧮 European Call Option Pricing using Monte Carlo Simulation

This notebook uses a Monte Carlo approach to estimate the fair price of a European call option under the Black-Scholes framework.

We simulate thousands of potential future stock price paths, compute the payoff of the option at maturity, and discount those payoffs to present value.

---

**Model assumptions:**
- Constant volatility (Black-Scholes)
- Risk-neutral pricing
- European-style option (exercise only at maturity)


## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

## Parameters

In [2]:
# Option parameters
S0 = 100       # Initial stock price
K = 105        # Strike price
T = 1.0        # Time to maturity (in years)
r = 0.05       # Risk-free interest rate (annual)
sigma = 0.2    # Volatility of the underlying asset

# Simulation parameters
n_simulations = 10000  # Number of Monte Carlo paths